In [4]:
from math import log
import os
import pandas as pd

In [5]:
os.chdir("C:/Users/claud/Documents/GitHub/correcci-n_nombres")

In [6]:
#Detectar todos los archivos csv de la carpeta csv dentro del directorio de trabajo",
csv_files = [f for f in os.listdir("raw_data/csv") if f.endswith('.csv')]

In [7]:
#Leer todos los archivos csv y concatenarlos en un solo dataframe"
df = pd.concat((pd.read_csv("raw_data/csv/"+f, encoding='utf-8', sep=',', low_memory=False) for f in csv_files))
#Columnas en minuscúlas",
df.columns = map(str.lower, df.columns)
#Seleccionar columnas 3 a 5"
df = df.iloc[:, 3:6]
#Renombrar columnas"
df.columns = ["primer_apellido", "segundo_apellido", "nombre"]
#Eliminar los registros "MENOR" en las tres columnas usando un loop"
for i in df.columns:
    df = df[df[i] != "MENOR"]
    #Eliminar nans de todas las columnas",
    df = df.dropna()

#Dejar únicamente el primer nombre"
df["nombre"] = df["nombre"].str.split(" ").str[0]
#Crear una lista de apellidos y nombres"
apellidos = df.primer_apellido.tolist() + df.segundo_apellido.tolist()
nombres = df.nombre.tolist()
#Crear una lista tanto de apellidos como de nombres
apellidos_nombres = apellidos + nombres
#Imprmir len con comas
print("Se cuenta con {:,} registros de apellidos y nombres".format(len(apellidos_nombres)))

Se cuenta con 6,086,922 registros de apellidos y nombres


In [8]:
#Obtener el costo 
wordcost=dict((k,log((i+1)*log(len(apellidos_nombres)))) for i,k in enumerate(set(apellidos_nombres)))
#Obtener maxword
maxword = max(len(x) for x in apellidos_nombres)

In [9]:
def costo_palabra(word):
    return wordcost.get(word, 9e999)

In [10]:
def infer_spaces(s):
    "Uses dynamic programming to infer the location of spaces in a string without spaces."
    # Find the best match for the i first characters, assuming cost has
    # been built for the i-1 first characters. Returns a pair (match_cost, match_length).
    def best_match(i):
        candidates = enumerate(reversed(cost[max(0, i-maxword):i]))
        return min((c + costo_palabra(s[i-k-1:i]), k+1) for k,c in candidates)

    # Build the cost array.
    cost = [0]
    for i in range(1,len(s)+1):
        c,k = best_match(i)
        cost.append(c)

    # Backtrack to recover the minimal-cost string.
    out = []
    i = len(s)
    while i>0:
        c,k = best_match(i)
        assert c == cost[i]
        out.append(s[i-k:i])
        i -= k

    return " ".join(reversed(out))

In [11]:
#Probar la función
infer_spaces("VICTORMIRELES")

'VICTOR MIRELES'